# Task 1: Find the Largest Number

In [4]:
import qiskit

import matplotlib.pyplot as plt
import numpy as np

from qiskit import QuantumCircuit, Aer, assemble
from qiskit.extensions import UnitaryGate
from qiskit.visualization import plot_histogram

## Grover's Search Algorithm

As was suggested in the associated references I used Grover's search alogrithm to solve this problem.

Grover's search algorithm searches an unordered dataset in O(√N), an improvement over the classical O(N) time. This is achieved in 3 main steps:
1. Place the circuit in superpostion (by the use of hadamard gates)
1. Applying an oracle to invert the amplited/phase of the target value(s)
1. Reflecting about the mean, thereby making the target values(s) the most probable

Steps 2 and 3 are repeated √N times to find the correct value with a high probability (N being array size)

## My Implementation

Below is my implementation, split into 2 parts:
1. make_oracle function that takes the 2 values and creates and oracle that finds the larger one
1. find_larget_number function which uses grover's search alogorithm to find the larger value by treating the two values as an array with a value that needs to be found

In [125]:
def make_oracle(number_1, number_2):
    """
    The oracle is a 'black box'
    
    In grover's algorithm its job is to invert the phase of the value searched for, in
    this case the greator value
    """
    
    if number_1 > number_2:
        oracle_unitary = [[-1, 0, 0, 0],
                          [0, 1, 0, 0],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]
    else:
        oracle_unitary = [[1, 0, 0, 0],
                          [0, -1, 0, 0],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]
        
    return UnitaryGate(oracle_unitary, label='Oracle')

In [126]:
def find_the_largest_number(number_1, number_2):
    """
    Find the larger of 2 values using grover's search algorithm
    """
    
    # Initialise circuit
    qc = QuantumCircuit(2,1)
    
    # Place all qubits in superpostion
    qc.h(0)
    qc.h(1)
    
    # Apply oracle, inverting phase of target (larger) qubit
    qc.append(make_oracle(number_1, number_2), [0,1])
    
    # Flip about mean
    qc.h(0)
    qc.h(1)
    
    # Repeat to further amplify
    qc.append(make_oracle(number_1, number_2), [0,1])
    
    # Flip about mean
    qc.h(0)
    qc.h(1)
    
    # Measure
    qc.measure(0, 0)
    
    # Run simulation
    aer_sim = Aer.get_backend('aer_simulator')
    qobj = assemble(qc)
    result = aer_sim.run(qobj).result()
    counts = result.get_counts()
    
    # Use results to find largest value, treating inputted numbers as an array
    largest_value_index = int(counts.most_frequent())
    
    value_array = [number_1, number_2]
    
    return value_array[largest_value_index]

In [127]:
# Testing the function with some sample values
A = find_the_largest_number(5, -6)
print(A)

B = find_the_largest_number(-6, 5)
print(B)

C = find_the_largest_number(10, 100)
print(C)

C = find_the_largest_number(-10, -100)
print(C)

5
5
100
-10


## Explanation
Here is a circuit diagram:

In [130]:
qc = QuantumCircuit(2,1)

# Place all qubits in superpostion
qc.barrier(label='1.')
qc.h(0)
qc.h(1)

# Apply oracle, inverting phase of target (larger) qubit
qc.barrier(label='2.')
qc.append(make_oracle(0, 1), [0,1])

# Flip about mean
qc.barrier(label='3.')
qc.h(0)
qc.h(1)

# Repeat to further amplify
qc.barrier(label='Repeat')
qc.append(make_oracle(0, 1), [0,1])

# Flip about mean
qc.h(0)
qc.h(1)

# Measure
qc.measure(0, 0)
    
qc.draw()

1. ┌───┐ 2. ┌─────────┐ 3. ┌───┐ Repeat ┌─────────┐┌───┐┌─┐
q_0: ─░──┤ H ├─░──┤0        ├─░──┤ H ├───░────┤0        ├┤ H ├┤M├
      ░  ├───┤ ░  │  Oracle │ ░  ├───┤   ░    │  Oracle │├───┤└╥┘
q_1: ─░──┤ H ├─░──┤1        ├─░──┤ H ├───░────┤1        ├┤ H ├─╫─
      ░  └───┘ ░  └─────────┘ ░  └───┘   ░    └─────────┘└───┘ ║ 
c: 1/══════════════════════════════════════════════════════════╩═
                                                               0

Firstly the circuit is initiated as 2 qubits and a single classical bit (for measurement) and put into superpostion with Hadamard gates

Next the oracle is queried and supplies us with a unitary matrix that flips the amplitude of the value we're looking for. Its action can be described as:



Reflecting about the mean would not work with only one qubit as after applying the oracle its mean is 0, so there would be no probability change. To resolve this I introduced an ancilla qubit. This draws the probability away from 0 allowing the 'reflection' to function properly. 

Amplification is performed twice as, using the ancilla qubit, the simulated array size is 4, there by needing √4 repititions.


`f(|x>) = { -|x> if x = w 
          {  |x> if x ≠ w`
          
I'm looking forward to working & learning with you all! :)

### Sources
	A Quantum Algorithm for finding the Maximum: arXiv:quant-ph/9911082
    A Quantum Algorithm for Finding the Minimum: arXiv:quant-ph/9607014
    A fast quantum mechanical algorithm for database search: arXiv:quant-ph/9605043